In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from transformers import GPT2Model, GPT2Tokenizer
from sklearn.metrics import classification_report

# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

if device.type == 'cuda':
    print(f'Using device: {device} ({torch.cuda.get_device_name(0)})')

# Load the IMDb Movie Reviews dataset
data = pd.read_csv('data/IMDB Dataset.csv')

# Basic data cleaning function
def clean_text(text):  
    text = re.sub(r'[^a-zA-Z\s]', '', text)  
    return text.lower()

# Apply cleaning to the review column
data['cleaned_text'] = data['review'].apply(clean_text)

# Encode labels (positive: 1, negative: 0)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['sentiment'])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['label'], test_size=0.2, random_state=42)

# Tokenization for GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

max_length = 100

def encode_data(texts, labels, tokenizer, max_length):
    # Tokenize the texts
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    # Convert labels to tensor
    labels = torch.tensor(labels.values)
    return TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)

train_dataset = encode_data(X_train, y_train, tokenizer, max_length)
test_dataset = encode_data(X_test, y_test, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(1, x.size(0), 64).to(device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Hyperparameters
vocab_size = len(tokenizer)
embed_size = 64
hidden_size = 64
num_classes = 1

# Initialize and train the RNN model
rnn_model = RNNModel(vocab_size, embed_size, hidden_size, num_classes).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)

# Training loop for RNN
for epoch in range(5):
    rnn_model.train()
    for input_ids, attention_mask, labels in train_loader:
        input_ids, labels = input_ids.to(device), labels.to(device).float().unsqueeze(1)
        outputs = rnn_model(input_ids)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Save the RNN model
torch.save(rnn_model.state_dict(), 'rnn_model.pth')

# Evaluate the RNN model
def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for input_ids, attention_mask, labels in data_loader:
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            if isinstance(model, RNNModel):
                outputs = model(input_ids)
            else:
                outputs = model(input_ids, attention_mask)  # Pass both input_ids and attention_mask
            preds = torch.sigmoid(outputs).round()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

y_true, y_pred_rnn = evaluate_model(rnn_model, test_loader)
print("RNN Model Evaluation:")
print(classification_report(y_true, y_pred_rnn))

# Define the Hybrid Model (GPT-2 + CNN + RNN)
class HybridModel(nn.Module):
    def __init__(self, transformer_model, hidden_size, num_classes):
        super(HybridModel, self).__init__()
        self.transformer = transformer_model
        self.cnn = nn.Conv1d(in_channels=768, out_channels=64, kernel_size=3, padding=1)
        self.rnn = nn.RNN(64, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        transformer_output = self.transformer(input_ids, attention_mask=attention_mask).last_hidden_state
        transformer_output = transformer_output.permute(0, 2, 1)
        cnn_output = self.cnn(transformer_output)
        cnn_output = cnn_output.permute(0, 2, 1)
        h0 = torch.zeros(1, cnn_output.size(0), 64).to(device)
        rnn_output, _ = self.rnn(cnn_output, h0)
        out = self.fc(rnn_output[:, -1, :])
        return out

# Load pre-trained GPT-2 model
transformer_model = GPT2Model.from_pretrained('gpt2').to(device)

# Initialize and train the hybrid model
hybrid_model = HybridModel(transformer_model, hidden_size=64, num_classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(hybrid_model.parameters(), lr=0.001)

# Training loop for Hybrid Model
for epoch in range(5):
    hybrid_model.train()
    for input_ids, attention_mask, labels in train_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device).float().unsqueeze(1)
        outputs = hybrid_model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Save the Hybrid model
torch.save(hybrid_model.state_dict(), 'hybrid_model.pth')

# Evaluate the Hybrid model
y_true, y_pred_hybrid = evaluate_model(hybrid_model, test_loader)
print("Hybrid Model Evaluation:")
print(classification_report(y_true, y_pred_hybrid))

# Testing models with diverse reviews
def test_model(model, tokenizer, input_text):
    cleaned_text = clean_text(input_text)
    encoding = tokenizer(cleaned_text, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    input_ids, attention_mask = encoding['input_ids'].to(device), encoding['attention_mask'].to(device)
    model.eval()
    with torch.no_grad():
        if isinstance(model, RNNModel):
            output = model(input_ids)
        else:
            output = model(input_ids, attention_mask)
        prediction = torch.sigmoid(output).round().cpu().numpy()[0][0]
    sentiment = "Positive" if prediction == 1 else "Negative"
    return sentiment

# Load the RNN model
rnn_model = RNNModel(vocab_size, embed_size, hidden_size, num_classes).to(device)
rnn_model.load_state_dict(torch.load('rnn_model.pth'))

# Load the Hybrid model
hybrid_model = HybridModel(transformer_model, hidden_size=64, num_classes=1).to(device)
hybrid_model.load_state_dict(torch.load('hybrid_model.pth'))

# Test the models with a variety of reviews
reviews_to_test = [
    "This movie was fantastic! I loved every moment of it.",
    "I really didn't like this film. It was a waste of time.",
    "The plot was okay, but the acting was terrible.",
    "An absolute masterpiece! A must-watch for everyone.",
    "It was boring and too long. I wouldn't recommend it.",
    "The cinematography was beautiful, but the story fell flat.",
    "I enjoyed it, but it could have been better.",
    "What a terrible movie! I can't believe I wasted my money.",
    "A delightful experience! I would watch it again.",
    "Not my cup of tea, but I can see why others might like it."
]

for review in reviews_to_test:
    print("Testing RNN Model:")
    print(f"Review: {review} | Sentiment: {test_model(rnn_model, tokenizer, review)}")
    print("Testing Hybrid Model:")
    print(f"Review: {review} | Sentiment: {test_model(hybrid_model, tokenizer, review)}")


Using device: cuda
Using device: cuda (NVIDIA GeForce RTX 3050 6GB Laptop GPU)


C:\Users\HP\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Epoch 1, Loss: 0.6780561804771423
Epoch 2, Loss: 0.6881519556045532
Epoch 3, Loss: 0.637283205986023
Epoch 4, Loss: 0.5908198952674866
Epoch 5, Loss: 0.7011416554450989
RNN Model Evaluation:
              precision    recall  f1-score   support

           0       0.54      0.55      0.55      4961
           1       0.55      0.54      0.54      5039

    accuracy                           0.55     10000
   macro avg       0.55      0.55      0.55     10000
weighted avg       0.55      0.55      0.55     10000



C:\Users\HP\anaconda3\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch 1, Loss: 0.7061896324157715
Epoch 2, Loss: 0.6997522115707397
Epoch 3, Loss: 0.6761562824249268
Epoch 4, Loss: 0.6950558423995972
Epoch 5, Loss: 0.6951999068260193
Hybrid Model Evaluation:
              precision    recall  f1-score   support

           0       0.50      1.00      0.66      4961
           1       0.00      0.00      0.00      5039

    accuracy                           0.50     10000
   macro avg       0.25      0.50      0.33     10000
weighted avg       0.25      0.50      0.33     10000



C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Local\Temp\ipykernel_7004\272

Testing RNN Model:
Review: This movie was fantastic! I loved every moment of it. | Sentiment: Positive
Testing Hybrid Model:
Review: This movie was fantastic! I loved every moment of it. | Sentiment: Negative
Testing RNN Model:
Review: I really didn't like this film. It was a waste of time. | Sentiment: Negative
Testing Hybrid Model:
Review: I really didn't like this film. It was a waste of time. | Sentiment: Negative
Testing RNN Model:
Review: The plot was okay, but the acting was terrible. | Sentiment: Negative
Testing Hybrid Model:
Review: The plot was okay, but the acting was terrible. | Sentiment: Negative
Testing RNN Model:
Review: An absolute masterpiece! A must-watch for everyone. | Sentiment: Positive
Testing Hybrid Model:
Review: An absolute masterpiece! A must-watch for everyone. | Sentiment: Negative
Testing RNN Model:
Review: It was boring and too long. I wouldn't recommend it. | Sentiment: Positive
Testing Hybrid Model:
Review: It was boring and too long. I wouldn't recom